In [1]:
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

C:\Users\ankit\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\ankit\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("C:\\Users\\ankit\\Downloads\\Cirrhosis\\cirrhosis.csv")
df

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,1,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,2,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,3,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,4,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,5,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,414,681,D,NaN,24472,F,NaN,NaN,NaN,N,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,3.0
414,415,1103,C,NaN,14245,F,NaN,NaN,NaN,N,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,4.0
415,416,1055,C,NaN,20819,F,NaN,NaN,NaN,N,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,3.0
416,417,691,C,NaN,21185,F,NaN,NaN,NaN,N,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,3.0


In [3]:
features=df[['Age','Albumin']]
target = df['Status']
features

,Age,Albumin
0,21464,2.60
1,20617,4.14
2,25594,3.48
3,19994,2.54
4,13918,3.53
...,...,...
413,24472,2.96
414,14245,3.83
415,20819,3.42
416,21185,3.75


In [5]:
scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
pca = PCA(n_components=2)

In [6]:
def perform_clustering_and_evaluate(data, method):
    if method == 'kmeans':
        clustering = KMeans(n_clusters=2, random_state=42)
    elif method == 'kmeans++':
        clustering = KMeans(n_clusters=2, init='k-means++', random_state=42)
    elif method == 'hierarchical':
        clustering = AgglomerativeClustering(n_clusters=2)

    labels = clustering.fit_predict(data)

    silhouette = silhouette_score(data, labels)
    calinski_harabasz = calinski_harabasz_score(data, labels)
    davies_bouldin = davies_bouldin_score(data, labels)

    return silhouette, calinski_harabasz, davies_bouldin

In [7]:
results_kmeans = []
results_kplus = []
results_hierarchical = []

methods = ['kmeans', 'kmeans++', 'hierarchical']

In [8]:
for data, description in zip([features, scaler.fit_transform(features), min_max_scaler.fit_transform(features),
                              min_max_scaler.fit_transform(scaler.fit_transform(features)),
                              pca.fit_transform(min_max_scaler.fit_transform(scaler.fit_transform(features)))],
                             ['Original', 'Normalized', 'Transformed', 'Normalized + Transformed', 'Normalized + Transformed + PCA']):
    silhouette_kmeans, calinski_harabasz_kmeans, davies_bouldin_kmeans = perform_clustering_and_evaluate(data, 'kmeans')
    silhouette_kplus, calinski_harabasz_kplus, davies_bouldin_kplus = perform_clustering_and_evaluate(data, 'kmeans++')
    silhouette_hierarchical, calinski_harabasz_hierarchical, davies_bouldin_hierarchical = perform_clustering_and_evaluate(data, 'hierarchical')

    results_kmeans.append({
        'Preprocessing': description,
        'Silhouette': silhouette_kmeans,
        'Calinski-Harabasz': calinski_harabasz_kmeans,
        'Davies-Bouldin': davies_bouldin_kmeans
    },)

    results_kplus.append({
        'Preprocessing': description,
        'Silhouette': silhouette_kplus,
        'Calinski-Harabasz': calinski_harabasz_kplus,
        'Davies-Bouldin': davies_bouldin_kplus
    })

    results_hierarchical.append({
        'Preprocessing': description,
        'Silhouette': silhouette_hierarchical,
        'Calinski-Harabasz': calinski_harabasz_hierarchical,
        'Davies-Bouldin': davies_bouldin_hierarchical
    })
# Convert the list of dictionaries to a DataFrame
kmeans = pd.DataFrame(results_kmeans)
kplus = pd.DataFrame(results_kplus)
hierarchy = pd.DataFrame(results_hierarchical)

C:\Users\ankit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ankit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ankit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ankit\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of

In [9]:
kmeans.to_csv('C:\\Users\\ankit\\Downloads\\Cirrhosis\\102103035_kmeans.csv', index=False)
kplus.to_csv('C:\\Users\\ankit\\Downloads\\Cirrhosis\\102103035_kmeansplusplus.csv', index=False)
hierarchy.to_csv('C:\\Users\\ankit\\Downloads\\Cirrhosis\\102103035_hierarchial.csv', index=False)